<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/vector_drug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/drug/data 50 class add_augment.zip'

# image2vector

##224*224

###50cls

In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def process_images_in_folder(folder_path, output_csv_path):
    pdEmbedded = pd.DataFrame()
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedded = image_embedding(image_path)
        embedded['ID'] = image_file  # ใช้ชื่อไฟล์เป็น ID
        pdEmbedded = pd.concat([pdEmbedded, embedded], ignore_index=True)

    # สร้างโฟลเดอร์สำหรับ output_csv_path
    output_folder = os.path.dirname(output_csv_path)
    os.makedirs(output_folder, exist_ok=True)

    # บันทึก DataFrame ลงในไฟล์ CSV
    pdEmbedded.to_csv(output_csv_path, index=False)

# ฟังก์ชันสำหรับการวนลูปโฟลเดอร์
def process_all_folders(base_folder_path, output_base_folder):
    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        output_csv_path = os.path.join(output_base_folder, f"{subfolder_name}.csv")
        process_images_in_folder(subfolder, output_csv_path)
        print(f"Embedding data for {subfolder_name} has been saved to {output_csv_path}")

# ตัวอย่างการใช้งาน
base_folder_path = '/content/data 50 class add_augment - Copy'
output_base_folder = '/content/drug50cls'
os.makedirs(output_base_folder, exist_ok=True)  # สร้างโฟลเดอร์หลักถ้ายังไม่มี

process_all_folders(base_folder_path, output_base_folder)

###Cosine similarity

In [37]:
# ใช้โค้ดนี้
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/1.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

1/1 [==============================] - 2s 2s/step
Top 5 most similar classes:
1. Class: Amlopine5mg, Similarity Score: 0.7161690444865032
2. Class: Lanzaar100mg, Similarity Score: 0.6803849533189887
3. Class: Amlopine10mg, Similarity Score: 0.6692816362653489
4. Class: BlopressPlus8mg, Similarity Score: 0.6641728463302811
5. Class: Betalol10mg, Similarity Score: 0.6504445370409421


In [38]:
# ใช้โค้ดนี้
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/2.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

1/1 [==============================] - 1s 1s/step
Top 5 most similar classes:
1. Class: Amlopine5mg, Similarity Score: 0.879502767449559
2. Class: Lanzaar100mg, Similarity Score: 0.781758209956031
3. Class: Amlopine10mg, Similarity Score: 0.7680906590500388
4. Class: Prenolol100mg, Similarity Score: 0.7268558039314591
5. Class: BlopressPlus8mg, Similarity Score: 0.722209155683805


In [39]:
# ใช้โค้ดนี้
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/3.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

1/1 [==============================] - 1s 1s/step
Top 5 most similar classes:
1. Class: Amlopine5mg, Similarity Score: 0.9486079979071514
2. Class: Amlopine10mg, Similarity Score: 0.8104069798219041
3. Class: Januvia100mg, Similarity Score: 0.8086129175860868
4. Class: BlopressPlus8mg, Similarity Score: 0.8022745900427379
5. Class: Caduet10_10mg, Similarity Score: 0.798925025829272


##640*640

###50cls

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def process_images_in_folder(folder_path, output_csv_path):
    pdEmbedded = pd.DataFrame()
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedded = image_embedding(image_path)
        embedded['ID'] = image_file  # ใช้ชื่อไฟล์เป็น ID
        pdEmbedded = pd.concat([pdEmbedded, embedded], ignore_index=True)

    # สร้างโฟลเดอร์สำหรับ output_csv_path
    output_folder = os.path.dirname(output_csv_path)
    os.makedirs(output_folder, exist_ok=True)

    # บันทึก DataFrame ลงในไฟล์ CSV
    pdEmbedded.to_csv(output_csv_path, index=False)

# ฟังก์ชันสำหรับการวนลูปโฟลเดอร์
def process_all_folders(base_folder_path, output_base_folder):
    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        output_csv_path = os.path.join(output_base_folder, f"{subfolder_name}.csv")
        process_images_in_folder(subfolder, output_csv_path)
        print(f"Embedding data for {subfolder_name} has been saved to {output_csv_path}")

# ตัวอย่างการใช้งาน
base_folder_path = '/content/data 50 class add_augment - Copy'
output_base_folder = '/content/drug50cls_640'
os.makedirs(output_base_folder, exist_ok=True)  # สร้างโฟลเดอร์หลักถ้ายังไม่มี

process_all_folders(base_folder_path, output_base_folder)

###Cosine similarity

In [40]:
# ใช้โค้ดนี้
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/1.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

1/1 [==============================] - 4s 4s/step
Top 5 most similar classes:
1. Class: Janumet50_1000mg, Similarity Score: 0.7697937031176645
2. Class: Amlopine5mg, Similarity Score: 0.756331928649153
3. Class: BlopressPlus8mg, Similarity Score: 0.7547976022663783
4. Class: Amlopine10mg, Similarity Score: 0.7406243538814581
5. Class: Caduet10_10mg, Similarity Score: 0.7228856744360195


In [41]:
# ใช้โค้ดนี้
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/2.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

1/1 [==============================] - 2s 2s/step
Top 5 most similar classes:
1. Class: Amlopine5mg, Similarity Score: 0.9449585038147295
2. Class: Amlopine10mg, Similarity Score: 0.8903408344118724
3. Class: Betalol40mg, Similarity Score: 0.8847969044732318
4. Class: Prenolol50mg, Similarity Score: 0.8592386553476684
5. Class: Prenolol25mg, Similarity Score: 0.8568574919595107


In [ ]:
# ใช้โค้ดนี้
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:50]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/2.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

# บันทึกคลาสใหม่

In [50]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def save_embedding(embedding, class_name, output_folder):
    output_csv_path = os.path.join(output_folder, f"{class_name}.csv")
    embedding['ID'] = class_name  # ใช้ชื่อคลาสเป็น ID
    os.makedirs(output_folder, exist_ok=True)
    embedding.to_csv(output_csv_path, index=False)

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            all_embeddings[class_name] = df
    return all_embeddings

def find_most_similar_classes(new_embedding, all_embeddings):
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        embeddings_no_id = embeddings_df.drop(['ID'], axis=1)
        similarity_score = cosine_similarity(new_embedding, embeddings_no_id)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/1.jpg'

# แปลงภาพใหม่เป็น embedding
new_embedding = image_embedding(new_image_path)

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_embedding, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

# กำหนดเกณฑ์ความคล้ายคลึง
similarity_threshold = 0.8

# ตรวจสอบว่ามีคลาสที่คล้ายคลึงเกินเกณฑ์หรือไม่
if similar_classes[0][1] >= similarity_threshold:
    print(f"The new image is similar to an existing class: {similar_classes[0][0]}")
else:
    most_similar_class = similar_classes[0][0]
    most_similar_score = similar_classes[0][1]

    if most_similar_score > 0.7:  # เกณฑ์ที่ต่ำกว่า 0.8 แต่สูงพอที่จะพิจารณาว่าเป็นคลาสเดิม
        print(f"The new image is similar to the existing class: {most_similar_class} with a lower similarity score of {most_similar_score}")
    else:
        new_class_name = 'new_class_name'  # กำหนดชื่อคลาสใหม่
        new_class_folder = '/content/new_class_folder'  # โฟลเดอร์ใหม่สำหรับเก็บคลาสใหม่
        save_embedding(new_embedding, new_class_name, new_class_folder)
        print(f"The new image is considered a new class and saved as {new_class_name}.csv in {new_class_folder}")

1/1 [==============================] - 5s 5s/step
Top 5 most similar classes:
1. Class: Janumet50_1000mg, Similarity Score: 0.7697937031176645
2. Class: Amlopine5mg, Similarity Score: 0.756331928649153
3. Class: BlopressPlus8mg, Similarity Score: 0.7547976022663783
4. Class: Amlopine10mg, Similarity Score: 0.7406243538814581
5. Class: Caduet10_10mg, Similarity Score: 0.7228856744360195
The new image is similar to the existing class: Janumet50_1000mg with a lower similarity score of 0.7697937031176645


In [49]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def save_embedding(embedding, class_name, output_folder):
    output_csv_path = os.path.join(output_folder, f"{class_name}.csv")
    embedding['ID'] = class_name  # ใช้ชื่อคลาสเป็น ID
    os.makedirs(output_folder, exist_ok=True)
    embedding.to_csv(output_csv_path, index=False)

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            all_embeddings[class_name] = df
    return all_embeddings

def find_most_similar_classes(new_embedding, all_embeddings):
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        embeddings_no_id = embeddings_df.drop(['ID'], axis=1)
        similarity_score = cosine_similarity(new_embedding, embeddings_no_id)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/bakamol-10pill.jpg'

# แปลงภาพใหม่เป็น embedding
new_embedding = image_embedding(new_image_path)

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_embedding, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

# กำหนดเกณฑ์ความคล้ายคลึง
similarity_threshold = 0.8

# ตรวจสอบว่ามีคลาสที่คล้ายคลึงเกินเกณฑ์หรือไม่
if similar_classes[0][1] >= similarity_threshold:
    print(f"The new image is similar to an existing class: {similar_classes[0][0]}")
else:
    new_class_name = 'new_class_name'  # กำหนดชื่อคลาสใหม่
    new_class_folder = '/content/new_class_folder'  # โฟลเดอร์ใหม่สำหรับเก็บคลาสใหม่
    save_embedding(new_embedding, new_class_name, new_class_folder)
    print(f"The new image is considered a new class and saved as {new_class_name}.csv in {new_class_folder}")

1/1 [==============================] - 2s 2s/step
Top 5 most similar classes:
1. Class: Janumet50_1000mg, Similarity Score: 0.5996624659568887
2. Class: Micardis40mg, Similarity Score: 0.5861621568811083
3. Class: MicardisPlus80_12.5mg, Similarity Score: 0.5564659362471918
4. Class: Amlopine10mg, Similarity Score: 0.5546961790821457
5. Class: Lercadip20mg, Similarity Score: 0.5522643651388188
The new image is considered a new class and saved as new_class_name.csv in /content/new_class_folder


#โหลดข้อมูลเวกเตอร์

In [43]:
# Export Data drug50cls
# Define the folder you want to zip and download
import shutil

folder_to_download = '/content/drug50cls'

# Zip the folder
shutil.make_archive('/content/drive/MyDrive/drug/drug50cls', 'zip', folder_to_download)

# Download the zip file
from google.colab import files
files.download('/content/drive/MyDrive/drug/drug50cls.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
# Export Data drug50cls_640
# Define the folder you want to zip and download
import shutil

folder_to_download = '/content/drug50cls_640'

# Zip the folder
shutil.make_archive('/content/drive/MyDrive/drug/drug50cls_640', 'zip', folder_to_download)

# Download the zip file
from google.colab import files
files.download('/content/drive/MyDrive/drug/drug50cls_640.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>